## Bibliotecas

In [ ]:
import folium
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import math

## Grafo

In [48]:
class Vertice:
    
    def __init__(self, i): #Función constructor
        self.id = i #
        self.visitado = False
        self.nivel = -1
        self.vecinos = [] #Los vertices conectados a un vértice en particular (vecinos)

    def agregaVecinos(self, v): #V es el vecino a agregar
        if not v in self.vecinos: #Si v no está en la lista de vecinos se agrega
            self.vecinos.append(v) #v es el vecino a agregar

class Grafica: #Esta clase define a toda la grafica o grafo

    def __init__(self): #Función constructor
        self.vertices = {} #Se crea un diccionario de vertices
    
    def agregarVertice(self, v):
        if v not in self.vertices: #Si v no se encuentra en nuestros vertices
            #Se agrega el vertice al diccionario
            #Se crea una llave dentro del ciccionario
            #con el identificador v
            #Como valor almacena un objeto de tipo vertice con el id v
            #Ejemplo: vertices[12] -> objeto Vertice con id 12
            self.vertices[v] = Vertice(v)

    def agregarArista(self, a, b): #a y b son vértices conectados por la arista (origen y destino)
        if a in self.vertices and b in self.vertices: #Si los 2 vertices se encuentran en el disccionario
            self.vertices[a].agregaVecinos(b)
            self.vertices[b].agregaVecinos(a) #Se crea el camino entre a y b colocándolos como vecinos

## Lectura

In [ ]:
df = pd.read_csv('LimaStreets.csv')
df.columns = df.columns.str.strip()
df.head()
subset_of_df = df.sample(n=380)


some_map = folium.Map(location=[subset_of_df['Y'].mean(),subset_of_df['X'].mean()],zoom_start=10)


for row in subset_of_df.itertuples():
  some_map.add_child(folium.Marker(location=[row.Y,row.X], popup=row.ID))


some_map

In [ ]:
df.head()

In [ ]:
df = pd.read_csv('LimaStreets.csv')
df.columns = df.columns.str.strip()
df.head()
subset_of_df = df.sample(n=500)

g = Grafica()

for i in range(5000):
    g.agregarVertice(df.ORIGEN[i])

for i in range(5000):
    g.agregarArista(df.ORIGEN[i], df.DESTINO[i])

for v in g.vertices:
    print(v, g.vertices[v].vecinos)


## Grafo ilustrado

In [ ]:
def adjlShow():
  grafo = gv.Digraph("grafod3")
  n = len(df)
  for i in range(1500):
    grafo.edge(str(df.ORIGEN[i]),str(df.DESTINO[i]))
  return grafo

adjlShow()

## Algorítmo


In [56]:
import math

class Vertice:
	def __init__(self, i):
		self.id = i
		self.vecinos = []
		self.visitado = False
		self.padre = None
		self.costo = float('inf')
		self.x = float('inf')
		self.y = float('inf')

	def agregarVecino(self, v, p, x, y):
		#print(v)
		#if v not in self.vecinos:
		self.vecinos.append([v, p, x, y])

class Grafica:
	def __init__(self):
		self.vertices = {}

	def agregarVertice(self, id):
		if id not in self.vertices:
			self.vertices[id] = Vertice(id)

	def agregarArista(self, a, b, p, x, y):
		if a in self.vertices and b in self.vertices:
			self.vertices[a].agregarVecino(b, p, x, y)
			#self.vertices[b].agregarVecino(a, p)
			
	def camino(self, a, b):
		camino = []
		camino_x = []
		camino_y = []
		actual = b
		llegada = self.vertices[a].padre
		x = b
		y = b
		while actual != llegada:
			camino.insert(0, actual)
			x = self.vertices[actual].x
			camino_x.insert(0, x)
			y = self.vertices[actual].y
			camino_y.insert(0, y)
			actual = self.vertices[actual].padre
		return [camino, self.vertices[b].costo, camino_x, camino_y]

	def minimo(self, l):
		if len(l) > 0:
			m = self.vertices[l[0]].costo
			v = l[0]
			for e in l:
				if m > self.vertices[e].costo:
					m = self.vertices[e].costo
					v = e
			return v
		return None

	def dijkstra(self, a):
		if a in self.vertices:
			# 1 y 2
			self.vertices[a].costo = 0
			actual = a
			noVisitados = []
			
			for v in self.vertices:
				if v != a:
					self.vertices[v].costo = float('inf')
					#self.vertices[v].x = float('inf')
					#self.vertices[v].y = float('inf')
				self.vertices[v].padre = None
				noVisitados.append(v)

			while len(noVisitados) > 0:
				#3
				for vec in self.vertices[actual].vecinos:
					if self.vertices[vec[0]].visitado == False:
						# 3.a
						if self.vertices[actual].costo + vec[1] < self.vertices[vec[0]].costo:
							#print(vec[2])
							self.vertices[vec[0]].costo = self.vertices[actual].costo + vec[1]
							self.vertices[vec[0]].padre = actual
							self.vertices[actual].x = vec[2]
							self.vertices[actual].y = vec[3]
							#print(self.vertices[vec[0]].x)

				# 4
				self.vertices[actual].visitado = True
				noVisitados.remove(actual)

				# 5 y 6
				actual = self.minimo(noVisitados)
		else:
			return False

g = Grafica()
df = pd.read_csv('LimaStreets.csv')
for i in range(5000):
  g.agregarVertice(df.ORIGEN[i])
for i in range(5000):
  g.agregarArista(df.ORIGEN[i], df.DESTINO[i], df.DISTANCIA[i], df.X[i], df.Y[i])

g.dijkstra(1)

In [ ]:
origen = input("Ingrese el id Origen: ")
destino = input("Ingrese el id Destino: ")
camino = g.camino(int(origen), int(destino))
print(camino)
original_camino = camino[0]
peso = camino[1]
for i in range(len(original_camino)):
	print(camino[2][i])
	print(camino[3][i])



some_map = folium.Map(location=[camino[3][0].mean(),camino[2][0].mean()],zoom_start=100)

numeros=[1,2,3,4,5]
for i in range(len(camino[0])):
  some_map.add_child(folium.Marker(location=[camino[3][i],camino[2][i]], popup=str(camino[0][i])))

some_map